# Title

This Notebook...

## Initialization

### Properties

In [ ]:
date = "20171120"
roadNumber = 2
roadsFileName = "../tests/data/BPS_20171120.txt"
detectionsFileName = "../tests/data/A2_20171120.txt"
outputDirectory = "../tests/data"
speedThreshold = 65
flowThreshold = 40
spaceSmoothing = 10
timeSmoothing = 20
marginSpace = spaceSmoothing  # probably always the same
marginTime = timeSmoothing  # probably always the same

### Imports

In [ ]:
import logging

import numpy
import matplotlib.pyplot

import sys
sys.path.append("../")
import congestionfinder
import congestionfinder.bpsdetector
import congestionfinder.road
import congestionfinder.detection
import congestionfinder.speedflow
import congestionfinder.congestion
import patchfinder.patch

### Logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

## Roads

In [ ]:
bpsCodes = congestionfinder.bpsdetector.readCSVToBPSCodes(roadsFileName)

In [ ]:
for bpsCode in bpsCodes:
    print(bpsCode)

In [ ]:
roads = congestionfinder.road.parseBPSCodesToRoads(bpsCodes)

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

## Speed and Flows

In [ ]:
detections = congestionfinder.detection.readCSVToDetections(detectionsFileName)

In [ ]:
iterator = iter(detections)
for i in range(min(len(detections), 300)):
    detection = next(iterator)
    print(detection)

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = congestionfinder.speedflow.parseDetectionsToSpeedsAndFlows(detections, roads[roadNumber])

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = congestionfinder.speedflow.removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = congestionfinder.speedflow.removeLowFlowTimes(speedsWorkingDetectors, flowsWorkingDetectors) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
congestions = congestionfinder.congestion.parseSpeedFlowsToCongestions(speedsHighFlow, flowsHighFlow, speedThreshold, flowThreshold)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsWithoutMissingValues = congestionfinder.congestion.interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestionsSmoothed = congestionfinder.congestion.applySmoothingFilter(congestionsWithoutMissingValues, spaceSmoothing, timeSmoothing)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Filter

In [ ]:
congestionsBoolean = congestionsSmoothed < 1

In [ ]:
congestionPatches = patchfinder.patch.findPatches(congestionsBoolean)

In [ ]:
for congestionPatch in congestionPatches:
    print(congestionPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(congestionsSmoothed, congestionPatches)
congestionfinder.congestion.plotCongestionsWithPatches(congestionsBoolean, congestionPatches)

In [ ]:
congestionPatchesFiltered = congestionfinder.congestion.filterLargeCongestions(congestionPatches)

In [ ]:
for congestionPatch in congestionPatchesFiltered:
    print(congestionPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(congestionsSmoothed, congestionPatchesFiltered)
congestionfinder.congestion.plotCongestionsWithPatches(congestionsBoolean, congestionPatchesFiltered)

In [ ]:
maxSpaceIndex = len(maskWorkingDetectors[maskWorkingDetectors])  # TODO: Improve this
maxTimeIndex = len(maskHighFlow[maskHighFlow])  # TODO: Improve this
congestionPatchesWithMargins = congestionfinder.congestion.addMargins(congestionPatchesFiltered, marginSpace, marginTime, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex)

In [ ]:
for congestionPatch in congestionPatchesWithMargins:
    print(congestionPatch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(congestionsSmoothed, congestionPatchesWithMargins)
congestionfinder.congestion.plotCongestionsWithPatches(congestionsBoolean, congestionPatchesWithMargins)

In [ ]:
speedFlowPatches = congestionfinder.speedflow.unmaskPatches(congestionPatchesWithMargins, maskWorkingDetectors, maskHighFlow)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(speeds, speedFlowPatches)

In [ ]:
congestionfinder.speedflow.writeSpeedsAndFlowsToCSV(speeds, flows, speedFlowPatches, outputDirectory, date, roadNumber)

End of Notebook